### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _asadm_.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [7]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed.

In [8]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [9]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>7.2.1</version>
  </dependency>
</dependencies>

####  Java Client Imports and Test code

In [10]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;

import com.aerospike.client.cdt.ListPolicy;
import com.aerospike.client.cdt.ListOperation;
import com.aerospike.client.cdt.ListOrder;
import com.aerospike.client.cdt.ListWriteFlags;

import java.util.HashMap;
import java.util.Map;

import java.util.Random;

AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);
Key key_uo = new Key("test", "testset", "keyUnordered");
Key key_o = new Key("test", "testset", "keyOrdered");

client.delete(null, key_uo);
client.delete(null, key_o);

WritePolicy wPolicy = new WritePolicy();
ListPolicy lPolicy_o = new ListPolicy(ListOrder.ORDERED, ListWriteFlags.DEFAULT);

// Map Entry
Map<Value, Value> employeeMap = new HashMap<Value, Value>();
int iSalary = 0;
int iLevel = 0;
List<Value> outerList = new ArrayList<Value>();

// Let's add 10 inner list item in a record's list bin
for (int i = 0; i < 10; i++) {    
    //map item
    Random rSalary = new Random();
    Random rLevel = new Random();
    iSalary = (rSalary.nextInt(4)+3) * 1000;
    iLevel = rLevel.nextInt(10);
    employeeMap.put(Value.get("id"), Value.get(100+i));
    employeeMap.put(Value.get("name"), Value.get("user"+i));
    employeeMap.put(Value.get("salary"), Value.get(iSalary));
    employeeMap.put(Value.get("level"), Value.get(iLevel));

    List<Value> innerList = new ArrayList<Value>();
    innerList.add(Value.get(iSalary));
    innerList.add(Value.get(iLevel));
    innerList.add(Value.get(employeeMap));
    
    //Write UNORDERED (Default ListPolicy) to key_uo
    client.operate(wPolicy, key_uo,                  
                   ListOperation.append("data", Value.get(innerList), null)                
                  );
    
    //Write ORDERED (using lPolicy_o) to key_o
    client.operate(wPolicy, key_o,                  
                   ListOperation.append(lPolicy_o, "data", Value.get(innerList), null)                
                  );
    
                  
}   
//Check both records.
System.out.println("UNORDERED List: "+ client.get(null, key_uo)); 
System.out.println("ORDERED List: "+ client.get(null, key_o));  //Inserts ordered.


UNORDERED List: (gen:10),(exp:450852588),(bins:(data:[[3000, 7, {name=user0, id=100, salary=3000, level=7}], [5000, 2, {name=user1, id=101, salary=5000, level=2}], [5000, 2, {name=user2, id=102, salary=5000, level=2}], [5000, 3, {name=user3, id=103, salary=5000, level=3}], [3000, 7, {name=user4, id=104, salary=3000, level=7}], [5000, 2, {name=user5, id=105, salary=5000, level=2}], [4000, 4, {name=user6, id=106, salary=4000, level=4}], [6000, 7, {name=user7, id=107, salary=6000, level=7}], [5000, 5, {name=user8, id=108, salary=5000, level=5}], [4000, 6, {name=user9, id=109, salary=4000, level=6}]]))
ORDERED List: (gen:10),(exp:450852588),(bins:(data:[[3000, 7, {name=user0, id=100, salary=3000, level=7}], [3000, 7, {name=user4, id=104, salary=3000, level=7}], [4000, 4, {name=user6, id=106, salary=4000, level=4}], [4000, 6, {name=user9, id=109, salary=4000, level=6}], [5000, 2, {name=user1, id=101, salary=5000, level=2}], [5000, 2, {name=user2, id=102, salary=5000, level=2}], [5000, 2, {n

# Cleanup

In [11]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"